In [10]:
import utils

In [35]:
from src.NN.KerasMultiHead import MultiHeadModel
from src.Data.IMULocationDataset import IMULocationDataset
from src.Data.MergedIMULocationDataset import MergedIMULocationDataset
from whar_datasets import WHARDatasetID
from src.NN.Arch.ClassifierHead import *
from src.NN.Arch.ReconstructionHead import *
from src.NN.Arch.Encoder import *
from src.Data.SensorTypes import SensorLocation

import numpy as np

In [38]:
mhealth = IMULocationDataset(WHARDatasetID.MHEALTH)
dsads = IMULocationDataset(WHARDatasetID.DSADS)
pamap2 = IMULocationDataset(WHARDatasetID.PAMAP2)

Loaded IMU Dataset from cache.
Loaded IMU Dataset from cache.
Loaded IMU Dataset from cache.


In [30]:
merged_dataset = MergedIMULocationDataset([mhealth, dsads], locations=[SensorLocation.ARMS])

In [31]:
merged_dataset.train.samples.shape
ds_slice = merged_dataset.train.samples[0:5, :, :]

print(np.unique(merged_dataset.train.labels))

[ 8  9 10 12 13 15 18 19 20 21 22 24 26 32 36 37 38 40 43 44 51 52 55 56
 57]


In [32]:
from keras.models import Model
import tensorflow as tf
from keras import layers as L


backbone = build_conv_backbone_seq(ts_len=merged_dataset.train.samples.shape[1])
classifier_head = build_classifier_head(num_classes=merged_dataset.num_classes, channels=backbone.output_shape[-1])
backbone(ds_slice).shape
print("Backbone parameters:", backbone.count_params())
print("Classifier head parameters:", classifier_head.count_params())

multi_head = MultiHeadModel(backbone)
multi_head.add_head("classifier", classifier_head)
multi_head(ds_slice)["classifier"].shape

multi_head.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0),
    loss={"classifier": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={"classifier": [tf.keras.metrics.SparseCategoricalAccuracy(name="acc")]},
)

history = multi_head.fit(x=merged_dataset.train.samples, y={"classifier": merged_dataset.train.labels}, epochs=5, verbose=1)

Backbone parameters: 287634
Classifier head parameters: 7611
Epoch 1/5
1270/1275 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - acc: 0.5000 - loss: 1.7145

2026-01-09 15:04:08.026585: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1275/1275 ━━━━━━━━━━━━━━━━━━━━ 45s 20ms/step - acc: 0.5874 - loss: 1.3158
Epoch 2/5
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - acc: 0.7007 - loss: 0.9027
Epoch 3/5
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - acc: 0.7372 - loss: 0.7754
Epoch 4/5
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - acc: 0.7667 - loss: 0.6878
Epoch 5/5
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - acc: 0.7847 - loss: 0.6278


In [33]:
# def build_classifier_head(num_classes):
#     z_in = tf.keras.Input(shape=(128,), name="embedding")
#     x = L.Dense(64, activation="swish", name="clf_dense1")(z_in)
#     x = L.Dropout(0.3, name="clf_do1")(x)
#     x = L.Dense(num_classes, activation="softmax", name="clf_out")(x)
#     return tf.keras.Model(z_in, x, name="classifier_head")

# classifier_head = build_classifier_head(num_classes=dataset.num_classes)

In [37]:
from src.Data.Preprocessing import make_masked_reconstruction_ds

multi_head.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
    loss={"reconstruction": tf.keras.losses.MeanSquaredError()},
)

train_ds = make_masked_reconstruction_ds(merged_dataset.train.samples, batch_size=64, mask_ratio=0.2)
val_ds   = make_masked_reconstruction_ds(merged_dataset.val.samples,   batch_size=64, mask_ratio=0.2, shuffle=0)

multi_head.fit(train_ds, validation_data=val_ds, epochs=20, verbose=1)

Epoch 1/20


2026-01-09 15:15:17.005460: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-01-09 15:15:17.027936: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


638/638 ━━━━━━━━━━━━━━━━━━━━ 49s 40ms/step - loss: 0.1410 - val_loss: 0.1242
Epoch 2/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1211 - val_loss: 0.1189
Epoch 3/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1181 - val_loss: 0.1167
Epoch 4/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1161 - val_loss: 0.1150
Epoch 5/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1152 - val_loss: 0.1141
Epoch 6/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1139 - val_loss: 0.1134
Epoch 7/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1133 - val_loss: 0.1120
Epoch 8/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1125 - val_loss: 0.1123
Epoch 9/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1116 - val_loss: 0.1106
Epoch 10/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1116 - val_loss: 0.1107
Epoch 11/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.1110 - val_loss: 0.1096
Epoch 12/20
638/638 ━━━━━━━━━━━━━━━━━━━━ 

In [48]:
multi_head.backbone.save("pretrained_backbone.keras")

In [50]:
fine_tune_dataset = MergedIMULocationDataset([dsads, mhealth], locations=[SensorLocation.ARMS], ratio=0.1)

In [51]:
import keras

classifier_head = build_classifier_head(
    num_classes=merged_dataset.num_classes,
    channels=multi_head.backbone.output_shape[-1],
)

multi_head = MultiHeadModel(keras.models.load_model("pretrained_backbone.keras"))


multi_head.add_head("classifier", classifier_head)
# multi_head.backbone.trainable = False

multi_head.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=1.0),
    loss={"classifier": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={"classifier": [tf.keras.metrics.SparseCategoricalAccuracy(name="acc")]},
)

history = multi_head.fit(
    x=fine_tune_dataset.train.samples,
    y={"classifier": fine_tune_dataset.train.labels},
    validation_data=(fine_tune_dataset.val.samples, {"classifier": fine_tune_dataset.val.labels}),
    epochs=20,
    verbose=1,
)

Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step - acc: 0.2832 - loss: 2.7781 - val_acc: 0.4279 - val_loss: 1.9596
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.4272 - loss: 1.8920 - val_acc: 0.4998 - val_loss: 1.6652
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.4941 - loss: 1.6823 - val_acc: 0.5466 - val_loss: 1.5205
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5265 - loss: 1.5407 - val_acc: 0.5646 - val_loss: 1.4314
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5615 - loss: 1.4600 - val_acc: 0.5947 - val_loss: 1.3658
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5647 - loss: 1.4009 - val_acc: 0.5867 - val_loss: 1.3285
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5866 - loss: 1.3517 - val_acc: 0.5981 - val_loss: 1.2900
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - acc: 0.5900 - loss: 1.3273 - val_acc: 0.6137 - val_loss: 1.2615
Epoch 9/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms

In [ ]:
from src.Visualization.tsne import tsne, plot_tsne
import numpy as np
import keras
keras.models.load_model("pretrained_backbone.keras")

# If your backbone outputs (N,T,C), pool to (N,C) first:
feat = backbone.predict(merged_dataset.train.samples, batch_size=32, verbose=1).numpy()   # (N,T,C)
E = feat.mean(axis=1)
print(E.shape)

# res = tsne(E, perplexity=30, random_state=0)
# plot_tsne(res.xy, labels=merged_dataset.val.labels, title="Backbone embeddings (val)")


ModuleNotFoundError: No module named 'src'